In [ ]:
import streamlit as st
import pandas as pd
from datetime import datetime, timedelta,time
from streamlit_gsheets import GSheetsConnection
from auxiliar.auxiliar import *

In [ ]:
data_seletor = st.date_input(
    "Selecione a data",
    (three_days_ago, today),
    format="DD/MM/YYYY",
)

if len(data_seletor) == 2:
    data_inicial = datetime.combine(data_seletor[0], time.min)
    data_final = datetime.combine(data_seletor[1], time.max)
else:
    data_inicial = datetime.combine(data_seletor[0], time.min)
    data_final = datetime.combine(data_seletor[0], time.max)

query = {
    "billcharge_paidAt": {
        "$gte": data_inicial,
        "$lte": data_final
    }
}

os_processados = pegar_dados_mongodb("os_processados",query=query)

In [4]:
os_processados.columns

Index(['quote_id', 'billCharge_id', 'customer_id', 'customer_name',
       'store_name', 'quote_status', 'paymentMethod_name', 'billcharge_paidAt',
       'bill_installmentsQuantity', 'bill_amount', 'servicos_json', 'os_id',
       'id_conta_corrente', 'dados_cliente', 'isPaid', 'Tipo de Pagamento',
       'billcharge_dueAt', 'amount', 'os_na_base',
       'Selecionar notas para subir'],
      dtype='object')